In [49]:
from pygam import LinearGAM, s, f, l
import pandas as pd
import patsy as pt
import numpy as np
from plotly import subplots
import plotly.offline as py
import plotly.graph_objs as go

# Prep the dataset
#   Put this back on one line!!
data = pd.read_csv( "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv" )

In [50]:
# Generate x and y matrices
eqn = """trips ~ -1 + month + day + 
      hour"""
y,x = pt.dmatrices( eqn, data=data )

# Initialize and fit the model
model = LinearGAM( s(0) + s(1) + s(2) )
modelFit = model.gridsearch( np.asarray(x), y )

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:00
 18% (2 of 11) |####                     | Elapsed Time: 0:00:00 ETA:   0:00:01
 27% (3 of 11) |######                   | Elapsed Time: 0:00:00 ETA:   0:00:00
 36% (4 of 11) |#########                | Elapsed Time: 0:00:00 ETA:   0:00:00
 45% (5 of 11) |###########              | Elapsed Time: 0:00:00 ETA:   0:00:00
 54% (6 of 11) |#############            | Elapsed Time: 0:00:00 ETA:   0:00:00
 63% (7 of 11) |###############          | Elapsed Time: 0:00:00 ETA:   0:00:00
 72% (8 of 11) |##################       | Elapsed Time: 0:00:01 ETA:   0:00:00
 81% (9 of 11) |####################     | Elapsed Time: 0:00:01 ETA:   0:00:00
 90% (10 of 11) |#####################   | Elapsed Time: 0:00:01 ETA:   0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [51]:
# Specify plot titles and shape
titles = [ 'month', 'day', 'hour' ]

fig = subplots.make_subplots(rows = 1, cols = 3, 
	subplot_titles = titles)
fig['layout'].update(height = 800, width = 1200, 
	title='pyGAM', showlegend = False)

Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'month',
                     'x': 0.14444444444444446,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'day',
                     'x': 0.5,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'hour',
                     'x': 0.8555555555555556,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
         

In [52]:
for i, title in enumerate(titles):
  XX = modelFit.generate_X_grid(term=i)
  pdep, confi = modelFit.partial_dependence(term=i, width=.95)
  trace = go.Scatter(x=XX[:,i], y=pdep, mode='lines', 
  	name='Effect')
  ci1 = go.Scatter(x = XX[:,i], y=confi[:,0], 
  	line=dict(dash='dash', color='grey'), 
    	name='95% CI')
  ci2 = go.Scatter(x = XX[:,i], y=confi[:,1], 
  	line=dict(dash='dash', color='grey'), 
    name='95% CI')

  if i<3:
    fig.append_trace(trace, 1, i+1)
    fig.append_trace(ci1, 1, i+1)
    fig.append_trace(ci2, 1, i+1)
  else:
    fig.append_trace(trace, 2, i-2)
    fig.append_trace(ci1, 2, i-2)
    fig.append_trace(ci2, 2, i-2)
    
py.plot(fig)

'temp-plot.html'

In [55]:
dataNEW = pd.read_csv( "assignment_data_test.csv" )

#use the model to predict the number of trips using columns month, day, and hour
pred = modelFit.predict(dataNEW[['month', 'day', 'hour']])
